ユーティリティ

In [1]:
import requests
from tqdm import tqdm

def download_with_progress(url, filename):
    """Download file with progress bar"""
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

月の標高データはNASAが観測したデータを利用しやすいように加工して、誰でも利用できるように公開しています。
ここでは、[Moon LRO LOLA DEM 118m](https://astrogeology.usgs.gov/search/map/moon_lro_lola_dem_118m) を利用しました。
このデータはNASAの月探査機ルナー・リコネッサンス・オービター（LRO）搭載の月レーザー高度計（LOLA）のデータに基づいたデジタル標高モデル（DEM）です。

月面の標高データのURLと、それを保存するファイル名を指定して、ダウンロードを行います。
ただし、データの量も8GBと、少し大きめなので、ダウンロードには時間がかかり、データ処理をするPCにも相応の量のメモリが必要となります。
(すでにローカルフォルダにダウンロードしたデータがあれば、ダウンロードしません。)

In [2]:
import os

lunar_dem_url = "https://planetarymaps.usgs.gov/mosaic/Lunar_LRO_LOLA_Global_LDEM_118m_Mar2014.tif"
lunar_dem_file = os.path.join(os.getcwd(), 'data', os.path.basename(lunar_dem_url))
lunar_dem_file

# Download DEM file
if os.path.exists(lunar_dem_file):
    print("Data already downloaded.")
else:
    print("Downloading data...")
    download_with_progress(lunar_dem_url, lunar_dem_file)

Data already downloaded.


[Rasterio](https://github.com/rasterio/rasterio) を使用して月の標高モデルファイルを読み込みます。

Rasterio は、地形のラスタデータを読み書きするためのPythonパッケージです。ここで使う月面DEMのGeoTiffデータを読み込んで、そこに含まれるメタデータなどを利用できます。

In [3]:
import rasterio
import pprint

lunar_dem_dataset = rasterio.open(lunar_dem_file)

# Display metadata
pprint.pp(lunar_dem_dataset.meta)

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': -32768.0,
 'width': 92160,
 'height': 46080,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["SimpleCylindrical Moon",GEOGCS["GCS_Moon",DATUM["D_Moon",SPHEROID["Moon",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(118.4505876, 0.0, -5458203.076608,
       0.0, -118.4505876, 2729101.538304)}


読み込んだ月面DEMのデータを3Dモデルへ変換します。

月面DEMは標準半径の球面からの差分の値で記録されているので、基準の球面に凹凸をつけることで月面の地形を表現できます。

そのためにまず、3Dモデル処理パッケージの PyVista で月の平均半径（1,737,400メートル）の球（Sphere）のメッシュモデルを作成します。
次に、その球形のメッシュを構成する各点を月面DEMの標高データにしたがって移動してから、出力するモデルの大きさに縮小して新しいメッシュをつくります。

ここで、月面DEMの標高データをそのままの尺度で利用してしまうと、小さな立体モデルをつくったときにクレーターなどの盛り上がりが低すぎてよく見えません。
そこで、メッシュを変形させるときに変形の度合いを強くすることで、地形を強調できるようにしています。

In [4]:
import numpy as np
import math
import pyvista as pv
from rasterio.transform import rowcol

def map_data_on_sphere(data, transform, nodata, radius, lon_res, lat_res, data_scale):
    
    # 与えられた解像度で球体を作成
    sphere = pv.Sphere(radius=radius, theta_resolution=lon_res, phi_resolution=lat_res)
    
    # 球体の点からデータを取得し、データに従って出力モデルの点を変更
    points = sphere.points.copy()
    new_points = points.copy()
    
    for i, pt in enumerate(points):
        x, y, z = pt
        r = np.linalg.norm(pt) # 球体の中心(原点)からの距離を計算
        if r == 0:
            continue

        # 球体の表面の点に対応する地理座標(緯度、経度)を計算
        lat_deg = math.degrees(math.asin(z / r))
        lon_deg = math.degrees(math.atan2(y, x))

        # 地理座標(緯度、経度)[度]を投影座標系(x, y)[m]に変換
        # 単純な円筒投影を仮定している
        x_proj = radius * math.radians(lon_deg)
        y_proj = radius * math.radians(lat_deg)
        
        # 球体の表面の点に対応するデータの値を取得
        try:
            row, col = rowcol(transform, x_proj, y_proj)
            elev = data[row, col]
            if elev == nodata:
                elev = 0
        except Exception:
            elev = 0
        
        # データに従って出力モデルの点を計算する
        new_r = r + elev * data_scale # 元の半径にデータの値を加える
        factor = new_r / r # 元の半径と変異した半径との比率
        new_points[i] = pt * factor # 元の球体の点の座標と変異率で計算した出力モデルの点
    
    # 出力モデルの点を更新
    sphere.points = new_points
    return sphere


この関数を使って、3Dプリントに適した解像度や地形の強調度合いに調整して3Dモデルをつくります。

In [5]:
moon_radius=1737400 # 月の平均半径[m]
longitude_resolution = 360 * 4 # 経度方向の解像度
latitude_resolution = 180 * 4  # 緯度方向の解像度
data_scale = 2.0  # DEMの効果を誇張するために調整

# 球体にデータをマッピングする
lunar_surface = map_data_on_sphere(
    data=lunar_dem_dataset.read(1),
    transform=lunar_dem_dataset.transform,
    nodata=lunar_dem_dataset.nodata,
    radius=moon_radius,
    lon_res=longitude_resolution,
    lat_res=latitude_resolution,
    data_scale=data_scale,
    )


3Dモデルを表示してみます。

In [6]:
plotter = pv.Plotter()
plotter.add_mesh(lunar_surface, show_edges=False, color="white")
plotter.add_title("Lunar DEM\nres={} x {}, scale={}".format(longitude_resolution, latitude_resolution, data_scale))
plotter.show()

Widget(value='<iframe src="http://localhost:62068/index.html?ui=P_0x14e54efc0_0&reconnect=auto" class="pyvista…

次に、月面の色のデータを読み込みます。

月面の色のデータは、NASA の [CGI Moon Kit](https://svs.gsfc.nasa.gov/4720/) から取得しました。
このデータは、実測されたカラーデータの 7 つの波長帯域のうち 3 つを変更して組み合わせ、人間の目で見たものに近づけています。
ソースデータは、月面の 70°N から 70°S までをカバーしていて、残りの部分は単色のデータと標高データから補完されています。

ここで利用したのは数十cmの3Dモデルに十分な細かさの4kのデータです。
3Dモデルの厚さに利用するため、元のRGBカラーデータをモノクロに変換しておきます。

月面の色データをダウンロードします。
(すでにローカルフォルダにダウンロードしたデータがあれば、ダウンロードしません。)

In [7]:
import os
from PIL import Image
import pprint

lunar_color_url = "https://svs.gsfc.nasa.gov/vis/a000000/a004700/a004720/lroc_color_poles_4k.tif"
lunar_color_file = os.path.join(os.getcwd(), 'data', os.path.basename(lunar_color_url))

# Download color file
if os.path.exists(lunar_color_file):
    print("Data already downloaded.")
else:
    print("Downloading data...")
    download_with_progress(lunar_color_url, lunar_color_file)

color_data = Image.open(lunar_color_file)
pprint.pp(color_data)

if color_data.mode == "RGB":
    # モノクロ画像に変換
    color_data_gray = color_data.convert("L")

pprint.pp(color_data_gray)

Data already downloaded.
<PIL.TiffImagePlugin.TiffImageFile image mode=RGB size=4096x2048 at 0x301B65250>
<PIL.Image.Image image mode=L size=4096x2048 at 0x315121CD0>


読み込んだ月面の色のデータを球体の凹凸へ変換します。

与えられた表面のメッシュを構成する各点を月面の色のデータにしたがって移動して、メッシュを変形させます。
このとき、モノクロ色データが最大値を取ったときに元の表面からの変異を0にするように変形させます。
このようにすることで、明るいところが盛り上がり、暗いところが凹むような変形を行います。

In [ ]:
import numpy as np
import math
import pyvista as pv
from rasterio.transform import rowcol

def shade_on_surface(shade_data, transform, nodata, radius, surface, data_scale):
    """
    与えられた表面のメッシュデータを色データに従って変形させる関数
    Args:
        data (numpy.ndarray): データの配列
        transform (rasterio.transform.Transform): ラスターの変換
        nodata (int): データの欠損値
        radius (float): 実物の半径
        surface (mesh): 表面のメッシュデータ
        data_scale (float): データのスケール(凹凸の強調)
    Returns:
        mesh: 元の表面を変形させたメッシュデータ
    """

    shade_mesh = surface.copy()
    
    # maxとminの値を取得
    max_val = np.nanmax(shade_data)
    # min_val = np.nanmin(shade_data)
    
    # 表面の点からデータを取得し、データに従って出力モデルの点を変更
    points = shade_mesh.points.copy()
    new_points = points.copy()
    
    for i, pt in enumerate(points):
        x, y, z = pt
        r = np.linalg.norm(pt) # 原点からの距離を計算
        if r == 0:
            continue

        # 表面の点に対応する地理座標(緯度、経度)を計算
        lat_deg = math.degrees(math.asin(z / r))
        lon_deg = math.degrees(math.atan2(y, x))

        # 地理座標(緯度、経度)[度]を投影座標系(x, y)[m]に変換
        # 単純な円筒投影を仮定している
        x_proj = radius * math.radians(lon_deg)
        y_proj = radius * math.radians(lat_deg)
        
        # 表面の点に対応するデータの値を取得
        try:
            row, col = rowcol(transform, x_proj, y_proj)
            elev = shade_data[row, col]
            if elev == nodata:
                elev = 0
        except Exception:
            elev = 0

        # データに従って出力モデルの点を計算する
        new_r = r + elev * data_scale # 元の距離にデータの値を加える
        new_r = new_r - (max_val * data_scale) # データの最大値のときに、表面との距離を0にする
        factor = new_r / r # 元の距離と変異した距離との比率
        new_points[i] = pt * factor # 方向を変えずに、距離を変化させる
    
    # 出力モデルの点を更新
    shade_mesh.points = new_points
    return shade_mesh

色の平面画像データを月面へマップするために利用するアフィン変換を用意します。

In [9]:
from affine import Affine

lunar_color_data = np.array(color_data_gray)
image_width = color_data.width # pixels
image_height = color_data.height #pixels

# lunar_color_data = lunar_color_dataset.read(1)
# image_width = lunar_color_dataset.width # pixels
# image_height = lunar_color_dataset.height #pixels

moon_radius = 1737400  # meters
moon_circumference = 2 * math.pi * moon_radius
color_lon_res =  moon_circumference / image_width
color_lat_res = (moon_circumference / 2) / image_height
color_center_lon = moon_circumference / 2
color_center_lat = moon_circumference / 4
color_transform = Affine(color_lon_res, 0, color_center_lon,
                         0, - color_lat_res, color_center_lat)
color_transform

Affine(2665.138220872513, 0.0, 5458203.076346907,
       0.0, -2665.138220872513, 2729101.5381734534)

色の値を厚さへ変換する度合いを設定して、クレーターのある表面のメッシュモデルを色データで変形させたモデルを作成します。

In [ ]:
data_scale = 200.0  # change to exaggerate the data effect

lunar_shade_surface = shade_on_surface(
    shade_data=lunar_color_data, 
    transform=color_transform, 
    nodata=0, 
    radius=moon_radius,
    surface=lunar_surface,
    data_scale=data_scale,
    )

lunar_shade_surface

PolyData (0x315103d00)
  N Cells:    2067840
  N Points:   1033922
  N Strips:   0
  X Bounds:   -1.747e+06, 1.723e+06
  Y Bounds:   -1.739e+06, 1.719e+06
  Z Bounds:   -1.743e+06, 1.726e+06
  N Arrays:   1

3Dモデルを表示してみます。

In [11]:
plotter = pv.Plotter()
plotter.add_mesh(lunar_shade_surface, show_edges=False, opacity=1)
plotter.add_title("Lunar Color\nres={} x {}, scale={}".format(longitude_resolution, latitude_resolution, data_scale))
plotter.show()

Widget(value='<iframe src="http://localhost:62068/index.html?ui=P_0x31513ab10_1&reconnect=auto" class="pyvista…

```
longitude_resolution = 360 * 4
latitude_resolution = 180 * 4
```

この解像度で球体を作ると、点の数が100万個ほどになります。これは10cmくらいの3Dプリントとして出力するには多すぎます。

元の球体のメッシュは均一な大きさのセルで作られているので、平坦な地形に見られる不必要に細かいメッシュを間引きします。


元の月の色データの値

In [12]:
lunar_shade_surface

PolyData (0x315103d00)
  N Cells:    2067840
  N Points:   1033922
  N Strips:   0
  X Bounds:   -1.747e+06, 1.723e+06
  Y Bounds:   -1.739e+06, 1.719e+06
  Z Bounds:   -1.743e+06, 1.726e+06
  N Arrays:   1

元のメッシュデータを [pyvista.PolyDataFilters.decimate()](https://pyvista.github.io/pyvista-docs-dev-ja/api/core/_autosummary/pyvista.PolyDataFilters.decimate.html#pyvista.PolyDataFilters.decimate) を使って間引きします。

In [13]:
target_reduction = 0.90
print(f"Reducing {target_reduction * 100.0} percent out of the original mesh")
lunar_shade_surface_decimated = lunar_shade_surface.decimate(target_reduction)
lunar_shade_surface_decimated

Reducing 90.0 percent out of the original mesh


PolyData,Information
N Cells,206784
N Points,103420
N Strips,0
X Bounds,"-1.743e+06, 1.723e+06"
Y Bounds,"-1.737e+06, 1.720e+06"
Z Bounds,"-1.741e+06, 1.725e+06"
N Arrays,0


間引きした結果を3D表示して表面を見てみましょう。

In [14]:
plotter = pv.Plotter(shape=(1, 2))
plotter.add_mesh(lunar_shade_surface, show_edges=False, color="white")
plotter.add_title("Original\nCells={}\nPoints={}".format(lunar_shade_surface.n_cells, lunar_shade_surface.n_points))
plotter.subplot(0, 1)
plotter.add_mesh(lunar_shade_surface_decimated, show_edges=False, color="white")
plotter.add_title("decinate()\nCells={}\nPoints={}".format(lunar_shade_surface_decimated.n_cells, lunar_shade_surface_decimated.n_points))
plotter.show()

Widget(value='<iframe src="http://localhost:62068/index.html?ui=P_0x31513b110_2&reconnect=auto" class="pyvista…

月面も同じように間引きします。

In [15]:
target_reduction = 0.90
print(f"Reducing {target_reduction * 100.0} percent out of the original mesh")
lunar_surface_decimated = lunar_surface.decimate(target_reduction)
lunar_surface_decimated

Reducing 90.0 percent out of the original mesh


PolyData,Information
N Cells,206784
N Points,103446
N Strips,0
X Bounds,"-1.755e+06, 1.738e+06"
Y Bounds,"-1.753e+06, 1.740e+06"
Z Bounds,"-1.759e+06, 1.742e+06"
N Arrays,0


3Dプリントのために、サイズと厚みを指定して殻をつくります。

In [ ]:
from pymeshfix import MeshFix

model_radius = 0.05 # [m]
model_thickness = 0.0006 # [m]
hole_radius = 0.0115 # ランプを入れる穴の半径[m]

# 月面と月影のメッシュを結合して、ランプのシェルを作成
outer_mesh = lunar_surface_decimated.scale(model_radius / moon_radius, inplace=False)
outer_mesh = MeshFix(outer_mesh).mesh
inner_mesh = lunar_shade_surface_decimated.scale((model_radius - model_thickness)/ moon_radius, inplace=False)
inner_mesh.flip_normals() # 面を内側へ向ける
inner_mesh = MeshFix(inner_mesh).mesh
lunar_shell = inner_mesh.merge(outer_mesh)

# # Rotate around the y-axis
# rotation_angle = 20 # [degrees]
# lunar_shell.rotate_y(rotation_angle)

# ランプの穴を作成
hole_center = (0, 0, -model_radius)
cyl = pv.Cylinder(
    radius=hole_radius,
    height=model_radius,
    direction=(0, 0, -1),
    center=hole_center)
lunar_lamp = lunar_shell.triangulate(inplace=False)
cyl.triangulate(inplace=True)
lunar_lamp = lunar_lamp.boolean_difference(cyl)

# ランプを表示
plotter = pv.Plotter()
plotter.add_mesh(lunar_lamp, show_edges=False, color="white", opacity=0.9)
plotter.add_title("Lunar Lamp")
plotter.show()

Widget(value='<iframe src="http://localhost:62068/index.html?ui=P_0x3389a00e0_12&reconnect=auto" class="pyvist…

STLファイルとして保存します。

In [34]:
lunar_lamp.save("model/lunar_lamp.stl")